In [126]:
import pandas as pd
import numpy as np
import re

### Import datasets

In [156]:
tweets1 = pd.read_json("BLM_1.json")
tweets2 = pd.read_json("BLM_2.json")
tweets3 = pd.read_json("BLM_3.json")
tweets = tweets1.append([tweets2, tweets3])
tweets = tweets.sort_values(by='created_at')

In [157]:
tweets = tweets.reset_index(drop=True)
tweets['long'] = 0.
tweets['lat'] = 0.
for idx, val in enumerate(tweets['coordinates']):
        tweets['lat'][idx] = val['coordinates'][0]
        tweets['long'][idx] = val['coordinates'][1]
tweets = tweets.drop(columns=['coordinates'])

### From here we get lemmatize the text

In [159]:
def clean_hyperlinks(text):
    result = re.sub(r'https?://[^\s<>"]+|www\.[^\s<>"]+'," ", text)
    return result

In [160]:
#tweets = tweets[tweets['lang'] == 'en']
tweets['text'] = tweets['text'].apply(clean_hyperlinks)
#tweets['text'].replace('', np.nan, inplace=True)
#tweets.dropna(subset=['text'], inplace=True)

In [161]:
def string_manipulation(df,column): 
    #extract hashtags
    df["hashtag"]  = df[column].str.findall(r'#.*?(?=\s|$)')
    #extract twitter account references
    df["accounts"] = df[column].str.findall(r'@.*?(?=\s|$)')
    return df
tweets = string_manipulation(tweets,"text")

In [162]:
tweets['hashtag'] = tweets['hashtag'].apply(lambda x: ','.join(map(str, x)))
from string import punctuation
def punctation(text): 
    result = text.replace("#", "")
    result = result.lower()
    return result
tweets['hashtag'] = tweets['hashtag'].apply(punctation)

In [163]:
def inlist(text): 
    result = text.split(",")
    return result
tweets['hashtag'] = tweets['hashtag'].apply(inlist)

In [164]:
list_of_tweets = []
for tweet in tweets['text']:
    temp = tweet
    list_of_tweets.append(temp)

In [165]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

# Create a set of stopwords
stop = set(stopwords.words('english'))

# Create a set of punctuation words 
exclude = set(string.punctuation) 

# This is the function makeing the lemmatization
lemma = WordNetLemmatizer()

# In this function we perform the entire cleaning
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

# This is the clean corpus.
doc_clean = [clean(doc).split() for doc in list_of_tweets] 
tweets['token'] = doc_clean

In [166]:
tweets.head(5)

,created_at,id,text,lang,geo,long,lat,hashtag,accounts,token
0,2020-05-07 12:37:45+00:00,1258375465271607296,This is a disgrace #runwithmaud @kamalaharris ...,en,NaN,37.97810,-122.031000,"[runwithmaud, justiceforahmaudarbery, justice,...","[@kamalaharris, @]","[disgrace, runwithmaud, kamalaharris, justicef..."
1,2020-05-07 14:15:16+00:00,1258400009755770881,#blacklivesmatter #seanreed #ahmaudarbery #bla...,in,NaN,39.76920,-86.158200,"[blacklivesmatter, seanreed, ahmaudarbery, bla...",[@],"[blacklivesmatter, seanreed, ahmaudarbery, bla..."
2,2020-05-07 14:49:01+00:00,1258408502227349504,#whiteapathy laylafsaad #blacklivesmatter @ In...,in,NaN,39.76920,-86.158200,"[whiteapathy, blacklivesmatter]",[@],"[whiteapathy, laylafsaad, blacklivesmatter, in..."
3,2020-05-07 15:38:24+00:00,1258420928993562624,"#BlackLivesMatter #AhmaudArbery @ Everywhere, ...",en,NaN,31.81095,-94.195982,"[blacklivesmatter, ahmaudarbery]",[@],"[blacklivesmatter, ahmaudarbery, everywhere, usa]"
4,2020-05-07 16:14:08+00:00,1258429920780390412,Just incase you were no sure! This is what inf...,en,NaN,40.71420,-74.006400,"[whiteprivilege, truthofthematter, callathinga...",[@],"[incase, sure, influence, whiteprivilege, look..."


In [167]:
for idx, val in enumerate(tweets['created_at']):
    tweets['created_at'][idx] = str(tweets['created_at'][idx]).split()[0]

In [168]:
tweets.to_csv('tweets_BLM.csv')